In [1]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
import keras.utils
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

### 데이터 불러오기

- EDA 데이터 모델 적용 결과 feature importance를 통해 칼럼을 제거한 데이터를 사용하면 모델의 복잡도 감소 + val_accuracy는 유지 가능

In [2]:
# 새로운 데이터 (EDA)과정 진행 후 데이터
X_train = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/X_fi_train.csv')
X_test = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/X_fi_test.csv')
X_val = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/X_fi_val.csv')
y_train = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/y_fi_train.csv')
y_test = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/y_fi_test.csv')
y_val = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/y_fi_val.csv')

In [3]:
# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [4]:
y_train = y_train['class']
y_test = y_test['class']
y_val = y_val['class']

# 타겟 맵핑				
label_map = {'Pastry': 0, 'Z_Scratch': 1, 'K_Scatch': 2, 'Stains': 3, 'Dirtiness': 4, 'Bumps': 5, 'Other_Faults': 6}
y_train = [label_map[label] for label in y_train]
y_test = [label_map[label] for label in y_test]
y_val = [label_map[label] for label in y_val]

# y_train을 one-hot encoding으로 변환합니다
num_classes = len(set(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)


### 모델 구현
- 큰 노드부터 감소시키는 은닉층을 갖는 구조 + 출력층 활성화함수 'softmax'고정

In [14]:
# 복습한 내용으로 추가적인 진행 - 다이아몬드형
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(512, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(256, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가   
            model.add(Dense(96, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(64, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(16, activation=func))
            model.add(Dense(12, activation=func)) 
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(loss='categorical_crossentropy', optimizer=custom_opt(i), metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:',best_time)
print('Best time hyperparameters:',time_hyper)


10/10 - 0s - loss: 0.8046 - accuracy: 0.7203 - 45ms/epoch - 5ms/step
10/10 - 0s - loss: 0.8345 - accuracy: 0.7363 - 47ms/epoch - 5ms/step
10/10 - 0s - loss: 0.9288 - accuracy: 0.7428 - 55ms/epoch - 5ms/step
10/10 - 0s - loss: 1.0047 - accuracy: 0.7524 - 53ms/epoch - 5ms/step
10/10 - 0s - loss: 1.1451 - accuracy: 0.7235 - 53ms/epoch - 5ms/step
10/10 - 0s - loss: 0.7729 - accuracy: 0.7492 - 43ms/epoch - 4ms/step
10/10 - 0s - loss: 0.8225 - accuracy: 0.7331 - 46ms/epoch - 5ms/step
10/10 - 0s - loss: 0.8564 - accuracy: 0.7460 - 58ms/epoch - 6ms/step
10/10 - 0s - loss: 0.8371 - accuracy: 0.7395 - 50ms/epoch - 5ms/step
10/10 - 0s - loss: 1.0017 - accuracy: 0.7267 - 75ms/epoch - 8ms/step
10/10 - 0s - loss: 0.7727 - accuracy: 0.7556 - 43ms/epoch - 4ms/step
10/10 - 0s - loss: 0.7849 - accuracy: 0.7588 - 55ms/epoch - 6ms/step
10/10 - 0s - loss: 0.7824 - accuracy: 0.7717 - 46ms/epoch - 5ms/step
10/10 - 0s - loss: 0.8511 - accuracy: 0.7556 - 51ms/epoch - 5ms/step
10/10 - 0s - loss: 1.0224 - accura

### 최종 모델
- Best hyperparameters: {'activation': 'relu', 'learning_rate': 0.0005, 'batch': 64}
- Best validation accuracy: 0.7813504934310913

In [47]:
func = 'relu'

model = Sequential()
model.add(Dense(512, activation=func, input_dim=input_dim))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(256, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가              
model.add(Dense(128, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가   
model.add(Dense(96, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(64, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(28, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(28, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(output_dim, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=custom_opt(0.0005), metrics=['accuracy'])

# Early stopping 기능 추가
early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
start_time = time.time()

# 모델 학습
model.fit(X_train, y_train, epochs=1000, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])
end_time = time.time()
long_time = end_time - start_time

print("==== Final Model ====")
print("걸린시간 :", long_time)
print("==== train los acc ====")
train_loss, train_acc = model.evaluate(X_train, y_train, verbose = 2)
print("==== val los acc ====")
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)

Epoch 1/1000
20/20 [==============================] - 4s 20ms/step - loss: 2.3253 - accuracy: 0.1942 - val_loss: 1.9021 - val_accuracy: 0.2862
Epoch 2/1000
20/20 [==============================] - 0s 8ms/step - loss: 1.9570 - accuracy: 0.2812 - val_loss: 1.8496 - val_accuracy: 0.3537
Epoch 3/1000
20/20 [==============================] - 0s 8ms/step - loss: 1.7700 - accuracy: 0.3175 - val_loss: 1.8024 - val_accuracy: 0.4180
Epoch 4/1000
20/20 [==============================] - 0s 8ms/step - loss: 1.6759 - accuracy: 0.3650 - val_loss: 1.7440 - val_accuracy: 0.4920
Epoch 5/1000
20/20 [==============================] - 0s 8ms/step - loss: 1.5679 - accuracy: 0.3997 - val_loss: 1.6706 - val_accuracy: 0.5016
Epoch 6/1000
20/20 [==============================] - 0s 8ms/step - loss: 1.4558 - accuracy: 0.4553 - val_loss: 1.5832 - val_accuracy: 0.5305
Epoch 7/1000
20/20 [==============================] - 0s 8ms/step - loss: 1.4047 - accuracy: 0.4867 - val_loss: 1.4861 - val_accuracy: 0.5498
Epoch